# Facebook: Predicting Check Ins

### Group: Tinh Nguyen, Joelynn Deng, AC Dela Paz, Suman Tripathy

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn import datasets
from matplotlib.colors import ListedColormap
from sklearn.cross_validation import cross_val_score, train_test_split
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression, RidgeClassifier
np.random.seed(30)

In [ ]:
#APK Metric Function
def apk(actual, predicted, k=3):
    if len(predicted) > k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

In [ ]:
#Read in the data
fb_train = pd.read_csv("../../datasets/Facebook/train.csv")
fb_test = pd.read_csv("../../datasets/Facebook/test.csv")
fb_train.head()

X = fb_train.drop(['place_id'], axis=1)
y = fb_train['place_id']

In [3]:
#Processing 
initial_date = np.datetime64('2014-01-01T01:01',   #Arbitrary 
                                 dtype='datetime64[m]') 
d_times = pd.DatetimeIndex(initial_date + np.timedelta64(int(mn), 'm') 
                           for mn in fb_train.time.values)   

NameError: name 'fb_train' is not defined

In [ ]:
X = fb_train.drop(['place_id'], axis=1)
y = fb_train['place_id']
fb_train.groupby('place_id').mean()

#Simply use test_train_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

#Test, train, & split by time
split_t = 730000
X_traintime = X[X.time < split_t]
X_testtime = X[X.time >= split_t]
y_traintime = fb_train[fb_train.time < split_t]['place_id']
y_testtime = fb_train[fb_train.time >= split_t]['place_id']

In [4]:
#Create our grid
x_range = 0.5
y_range = 0.25

x_weights = 500
y_weights = 1000

xpoints = np.arange(0,x_weights*10,x_range*x_weights)
xpoints = np.append(xpoints,x_weights*10+x_range*x_weights)
ypoints = np.arange(0,y_weights*10,y_range*y_weights)
ypoints = np.append(ypoints,y_weights*10+y_range)
x_grid, y_grid = np.meshgrid(xpoints,ypoints)

In [ ]:
#Train on one grid using normal training set
score = 0.0
total = 0
cumulative_scores = []

i = 0
j = 0

subset = X_train[(X_train['x'] >= x_grid[i][j]) & (X_train['x'] < x_grid[i][j+1]) & \
                     (X_train['y'] >= y_grid[i][j]) & (X_train['y'] < y_grid[i+1][j])]
test_subset = X_test[(X_test['x'] >= x_grid[i][j]) & (X_test['x'] < x_grid[i][j+1]) & \
                         (X_test['y'] >= y_grid[i][j]) & (X_test['y'] < y_grid[i+1][j])]

In [36]:
#Train on one grid using time-based training set
time_score = 0.0
time_total = 0
time_cumulative_scores = []

i = 0
j = 0

time_subset = X_traintime[(X_traintime['x'] >= x_grid[i][j]) & (X_traintime['x'] < x_grid[i][j+1]) & \
                     (X_traintime['y'] >= y_grid[i][j]) & (X_traintime['y'] < y_grid[i+1][j])]
time_test_subset = X_testtime[(X_testtime['x'] >= x_grid[i][j]) & (X_testtime['x'] < x_grid[i][j+1]) & \
                         (X_testtime['y'] >= y_grid[i][j]) & (X_testtime['y'] < y_grid[i+1][j])]

7.580753568836803
7.249712698123841


In [54]:
X_train.head()

,row_id,x,y,accuracy,time
305587,305587,5.1050,6.4115,172,362163
93140,93140,2.6905,5.8470,5,369458
781106,781106,4.4467,9.9711,67,560596
1007270,1007270,6.8005,4.7053,57,110013
65266,65266,9.6079,4.5557,67,743654


In [ ]:
X_vals = (X_train['x'] > randomx) & (X_train['x'] < (randomx + .25))
Y_vals = (X_train['y'] > randomy) & (X_train['y'] < (randomy + .25))
true_vals = X_vals & Y_vals
true_vals.shape
new_data = X_train.where(true_vals)
new_data.head()

In [ ]:
def cv_stats(cv_score):
    """ 
    Takes in the output of cross_val_score
    Returns the mean and standard deviation in a readable format
    """
    mean = np.mean(cv_score)
    std = np.std(cv_score)
    return mean, std

# K Nearest Neighbors Model

In [6]:
clf = KNN(n_neighbors = 50)
clf.fit(X_train, y_train)
all_predictions = clf.predict_proba(X_test)

all_predictions.shape

NameError: name 'X_train' is not defined

In [ ]:
#Test on time-based training set

clf.fit(X_traintime, y_traintime)
all_time_predictions = clf.predict_proba(X_testtime)


# Linear Regression Model


In [ ]:
# Initialize linear regression
clf = linear_model.LinearRegression()
clf.fit(X_train, y_train)


In [ ]:
#Score Linear Regression based on normal training set

train_r1 = clf.score(X_train, y_train) # fill out linreg
test_r1 = clf.score(X_test, y_test)# fill out linreg

print("Train accuracy : ", train_r1)
print("Test accuracy : ", test_r1)

In [ ]:
y_pred = clf.predict(X_test) # predict on test set

fig, ax = plt.subplots()
ax.scatter(y_test, y_pred)
ax.plot([y.min(), y.max()], [y.min(), y.max()], 'r--', lw=4)
ax.set_xlabel('Test')
ax.set_ylabel('Predicted')
plt.title('Comparison of Actual vs Predicted Values')
plt.show()

#Logistic Regression Model


In [ ]:
# iterate through a range of penalty values and evaluate how well LogisticRegression does on average
penalty = 0.001
while penalty <= 10:
    clf = LogisticRegression(C = penalty)
    score = cross_val_score(clf, X_train, y_train, cv = 5)
    stats = cv_stats(score)
    print("For penalty={0}", penalty, "; Average={0}, STDEV={1}".format(*stats))
    penalty *= 10
    
# initialize and fit logistic regression
clf = linear_model.LogisticRegression()
clf.fit(X_train, y_train)

In [ ]:
train_r2 = clf.score(X_train, y_train) # score Logistic Regression on training set
test_r2 = clf.score(X_test, y_test) # score Logistic Regression on test set


print("Train accuracy : ", train_r2)
print("Test accuracy : ", test_r2)

# Ridge Regression Model


In [58]:
# iterate through a range of penalty values and evaluate how well RidgeClassifier does on average
penalty = 0.01
while penalty < 10:
    clf = RidgeClassifier(alpha = penalty)
    score = cross_val_score(clf, X_train, y_train, cv = 5)
    stats = cv_stats(score)
    print("For penalty = ", penalty, "Average, std = ", stats)
    penalty *= 10

NameError: name 'RidgeClassifier' is not defined

# Best Model
## K Nearest Neighbors

In [1]:
# initialize KNN here
clf = KNN(40)
clf.fit(X_train, y_train)

train_acc = clf.score(X_train, y_train) # score KNN on train set
test_acc = clf.score(X_test, y_test) # score KNN on test set


print("Train accuracy : ", train_acc)
print("Test accuracy : ", test_acc)

NameError: name 'KNN' is not defined